<a href="https://colab.research.google.com/github/leonardo3108/robustez-query/blob/main/code/C%C3%A1lculo_do_Desconto_ou_Ganho.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_noise_kind.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_search_context.csv -P data
! wget -nc https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_original_query.csv -P data

--2021-11-25 00:20:43--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/code/local/util_bd_dataframe.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12040 (12K) [text/plain]
Saving to: ‘util_bd_dataframe.py’

util_bd_dataframe.p 100%[===================>]  11.76K  --.-KB/s    in 0s      

2021-11-25 00:20:43 (130 MB/s) - ‘util_bd_dataframe.py’ saved [12040/12040]

--2021-11-25 00:20:43--  https://raw.githubusercontent.com/leonardo3108/robustez-query/main/data/tab_calculated_metric.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting re

In [3]:
import util_bd_dataframe  as util_bd_pandas

df_calculated_metric = util_bd_pandas.read_df_calculated_metric_with_label()[['search_context','cod_metric','noise_kind', 'value','cod_original_query']]
df_calculated_metric

,search_context,cod_metric,noise_kind,value,cod_original_query
0,TREC 2020 Judment-BM25,DCG@10,No changes in the text,1.746141,23849
1,TREC 2020 Judment-BM25,DCG@10,No changes in the text,7.000000,42255
2,TREC 2020 Judment-BM25,DCG@10,No changes in the text,24.412392,47210
3,TREC 2020 Judment-BM25,DCG@10,No changes in the text,7.315465,67316
4,TREC 2020 Judment-BM25,DCG@10,No changes in the text,3.014736,118440
...,...,...,...,...,...
4855,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.665893,1030303
4856,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.456374,1037496
4857,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.772823,1043135
4858,TREC 2020 Judment-ReRank@100,nDCG@10,With one typographical error,0.594450,1051399


In [4]:
df_calculated_metric['search_context'].value_counts()

TREC 2020 Judment-DPR           972
TREC 2020 Judment-BM25          972
TREC 2020 Full-BM25             972
TREC 2020 Judment-ReRank@100    972
TREC 2020 Full-ReRank@1000      972
Name: search_context, dtype: int64

In [5]:
df_calculated_metric['cod_metric'].value_counts()

nDCG@10    2430
DCG@10     2430
Name: cod_metric, dtype: int64

# Comparação Robustez


In [20]:
nDGC10_original = df_calculated_metric.query('cod_metric == "nDCG@10" and noise_kind == "No changes in the text"').set_index(['cod_original_query', 'search_context'])[['value']].to_dict()['value']
nDGC10_original

{(23849, 'TREC 2020 Full-BM25'): 0.009088684028421674,
 (23849, 'TREC 2020 Full-ReRank@1000'): 0.5247570718868744,
 (23849, 'TREC 2020 Judment-BM25'): 0.05490162180062975,
 (23849, 'TREC 2020 Judment-DPR'): 0.7209795539661995,
 (23849, 'TREC 2020 Judment-ReRank@100'): 0.6143437063013758,
 (42255, 'TREC 2020 Full-BM25'): 0.7946549217704771,
 (42255, 'TREC 2020 Full-ReRank@1000'): 0.9096843957527361,
 (42255, 'TREC 2020 Judment-BM25'): 0.37291464931616275,
 (42255, 'TREC 2020 Judment-DPR'): 0.7199775715968755,
 (42255, 'TREC 2020 Judment-ReRank@100'): 0.9413498539097771,
 (47210, 'TREC 2020 Full-BM25'): 0.6369734002380002,
 (47210, 'TREC 2020 Full-ReRank@1000'): 0.9499670029785341,
 (47210, 'TREC 2020 Judment-BM25'): 0.8662197873089202,
 (47210, 'TREC 2020 Judment-DPR'): 0.8208114684630632,
 (47210, 'TREC 2020 Judment-ReRank@100'): 0.9716549839975663,
 (67316, 'TREC 2020 Full-BM25'): 0.26071401217877443,
 (67316, 'TREC 2020 Full-ReRank@1000'): 0.6469095432547697,
 (67316, 'TREC 2020 Judm

In [26]:
def calcular_perda_ganho(cod_original_query, search_context, value):
    base = nDGC10_original[(cod_original_query, search_context)]
    if base:
        return value / base - 1
    return float("NaN")
calcular_perda_ganho(1030303, 'TREC 2020 Full-ReRank@1000', 0.676534)

-0.0456676209736564

In [42]:
df_dg_nDCG10 = df_calculated_metric.query('cod_metric == "nDCG@10"')[['search_context', 'noise_kind', 'cod_original_query']]
df_dg_nDCG10['cod_metric'] = 'DG:nDCG@10'
df_dg_nDCG10['value'] = df_calculated_metric.query('cod_metric == "nDCG@10"').apply(lambda row:calcular_perda_ganho(row.cod_original_query, row.search_context, row.value), axis = 1)
df_dg_nDCG10.dropna(inplace=True)
df_dg_nDCG10

,search_context,noise_kind,cod_original_query,cod_metric,value
54,TREC 2020 Judment-BM25,No changes in the text,23849,DG:nDCG@10,0.000000
55,TREC 2020 Judment-BM25,No changes in the text,42255,DG:nDCG@10,0.000000
56,TREC 2020 Judment-BM25,No changes in the text,47210,DG:nDCG@10,0.000000
57,TREC 2020 Judment-BM25,No changes in the text,67316,DG:nDCG@10,0.000000
58,TREC 2020 Judment-BM25,No changes in the text,118440,DG:nDCG@10,0.000000
...,...,...,...,...,...
4855,TREC 2020 Judment-ReRank@100,With one typographical error,1030303,DG:nDCG@10,0.023385
4856,TREC 2020 Judment-ReRank@100,With one typographical error,1037496,DG:nDCG@10,-0.506072
4857,TREC 2020 Judment-ReRank@100,With one typographical error,1043135,DG:nDCG@10,-0.001899
4858,TREC 2020 Judment-ReRank@100,With one typographical error,1051399,DG:nDCG@10,-0.279501


In [ ]:
### util_bd_pandas.save_calculated_metric(dict_val:dict, cod_search_context:int, cod_noise_kind:int)